In [1]:
!pip install gdown

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [ ]:
import gdown
import zipfile
import os

# Enlace de la carpeta comprimida en Google Drive (debe estar configurado como público o compartir el enlace)
url = 'https://drive.google.com/uc?id=1w2D-lwIvD53bDOqD6WlX81Ma2qERoOws'
output = 'Models.zip'

# Descargar la carpeta comprimida
gdown.download(url, output, quiet=False)

# Ruta del archivo zip que deseas descomprimir
archivo_zip = 'Models.zip'
# Ruta donde quieres extraer los archivos
destino = 'ModelLanguage/'

# Si el directorio de destino no existe, lo creamos
if not os.path.exists(destino):
    os.makedirs(destino)

# Descomprimir el archivo zip
with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    zip_ref.extractall(destino)

print(f"Archivo descomprimido en {destino}")



: 

In [ ]:
import gdown

# Enlace de la carpeta comprimida en Google Drive (debe estar configurado como público o compartir el enlace)
url = 'https://drive.google.com/uc?id=1uEUCSEZ4FhvkLZ3U2ZxXfTtemSNseRCd'
output = 'Models/modelo_naive_bayes.pkl'

# Descargar la carpeta comprimida
gdown.download(url, output, quiet=False)

In [5]:
import tensorflow as tf
import joblib
from langdetect import detect
import tensorflow_text

# Diccionario de mapeo de etiquetas numéricas a idiomas
etiquetas_a_idiomas = {
    0: 'spa', 1: 'eng', 2: 'deu', 3: 'por', 4: 'fra', 5: 'ita',
    6: 'pol', 7: 'swe', 8: 'ces', 9: 'nno', 10: 'otros'
}

# Cargar los modelos de traducción
def load_translation_models():
    models = {
        #"EN": tf.saved_model.load('Models/ModelLanguage/translatorES-EN'),
        "FR": tf.saved_model.load('Models/ModelLanguage/translatorES-FR'),
        "DE": tf.saved_model.load('Models/ModelLanguage/translatorES-DEhalf'),
        "IT": tf.saved_model.load('Models/ModelLanguage/translatorES-IT'),
        "PT": tf.saved_model.load('Models/ModelLanguage/translatorES-PT'),
    }
    return models

# Cargar el modelo de detección de idioma
def load_language_detection_model():
    return joblib.load('Models/modelo_naive_bayes.pkl')

# Función para detectar el idioma usando el modelo o langdetect
def detect_language(text, model):
    try:
        # Intentamos usar el modelo cargado para predecir el idioma
        predicted_language_idx = model.predict([text])[0]
        print(f"Predicción del modelo (índice): {predicted_language_idx}")
        
        # Mapeamos el índice a la etiqueta de idioma
        predicted_language = etiquetas_a_idiomas.get(predicted_language_idx, 'otros')
        
        # Si el idioma es uno de los soportados, lo devolvemos
        supported_languages = ['spa', 'eng', 'deu', 'por', 'fra', 'ita']
        if predicted_language in supported_languages:
            return predicted_language
        else:
            return 'otros'  # Devolver "otros" si el idioma no está en los soportados
    except:
        # Si el modelo no puede identificar el idioma, usamos langdetect
        detected_language = detect(text)
        return detected_language

# Función para traducir a español si el idioma detectado es uno de los soportados
def translate_to_spanish(text, model):
    return model(text).numpy().decode('utf-8')

# Función para traducir desde español a otro idioma
def translate_from_spanish(text, target_lang, models):
    if target_lang in models:
        model = models[target_lang]
        return model(text).numpy().decode('utf-8')
    else:
        return f"Lo siento, no hay opción para traducir a {target_lang}."

def main():
    # Cargar los modelos de traducción
    models = load_translation_models()
    print('Modelos de traducción cargados')
    
    # Cargar el modelo de detección de idioma
    language_model = load_language_detection_model()
    print('Modelo de detección de idioma cargado')
    
    # Solicitar frase al usuario
    text = input("Introduce una frase para traducir: ")
    
    # Detectar el idioma usando el modelo o langdetect
    detected_language = detect_language(text, language_model)
    print(f"Idioma detectado: {detected_language}")
    
    if detected_language == 'spa':
        # Si está en español, preguntar a qué idioma traducir
        target_language = input("¿A qué idioma deseas traducir? (EN, FR, DE, IT, PT): ").upper()
        translation = translate_from_spanish(text, target_language, models)
        print(f"Traducción: {translation}")
    elif detected_language in ['eng', 'fra', 'deu', 'por', 'ita']:
        # Si el idioma está en inglés, francés, alemán, italiano o portugués, traducir a español
        language_map = {'eng': 'EN', 'fra': 'FR', 'deu': 'DE', 'por': 'PT', 'ita': 'IT'}
        model = models[language_map[detected_language]]
        translation = translate_to_spanish(text, model)
        print(f"Traducción al español: {translation}")
    else:
        print(f"Lo siento, no tenemos soporte para el idioma detectado: {detected_language}.")

if __name__ == "__main__":
    main()


Modelos de traducción cargados
Modelo de detección de idioma cargado
Predicción del modelo (índice): 0
Idioma detectado: spa
Traducción: j ' ai froid et je veux un chandail .
